In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
import keras_tuner as kt

from tensorflow import keras
from keras.layers import Dense, Dropout

In [ ]:
print(tf.__version__)

# Function definition

In [ ]:
def plot_precision(h, p, v_p):
    plt.plot(h.history[p])
    plt.plot(h.history[v_p])
    plt.title('Model Precision')
    plt.ylabel('Precision')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

In [ ]:
def plot_loss(h):
    plt.plot(h.history['loss']) 
    plt.plot(h.history['val_loss']) 
    plt.title('Model loss') 
    plt.ylabel('Loss') 
    plt.xlabel('Epoch') 
    plt.legend(['Train', 'Test'], loc='upper left') 
    plt.show()

In [ ]:
def create_final_model(u, lr):
    
    inputs = keras.Input(shape=(_inputs, ))
    
    x = Dense(units = u,  activation="relu")(inputs)
    x = Dropout(0.5)(x)
    
    outputs = Dense(1, activation='sigmoid')(x)
    
    model = keras.Model(inputs, outputs, name="final_model")
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                   optimizer= keras.optimizers.Adam(learning_rate=lr),
                   metrics=[tf.keras.metrics.F1Score()],
                )
    model.summary()
    return model  

In [ ]:
class MyHyperModel(kt.HyperModel):
  def build(self, hp):
    
    # hp_units = hp.Int('units', min_value=32, max_value=round(_inputs/2), step=32)
    hp_units = hp.Int('units', min_value=round(_inputs/2), max_value=_inputs, step=142)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    inputs = keras.Input(shape=(_inputs, ))
    
    x = Dense(units = hp_units,  activation="relu")(inputs)
    x = Dropout(0.5)(x)
    
    outputs = Dense(1, activation='sigmoid')(x)
    
    model = keras.Model(inputs, outputs, name="model")
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                   optimizer= keras.optimizers.Adam(learning_rate=hp_learning_rate),
                   metrics=tf.keras.metrics.F1Score()
                )
    model.summary()
    return model


# Load data


In [ ]:
X_train = pd.read_csv('../data/processed/X_train.csv')
X_test = pd.read_csv('../data/processed/X_test.csv')

X_train.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
X_test.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

z_train = pd.read_csv('../data/processed/y_train.csv')
z_test = pd.read_csv('../data/processed/y_test.csv')

y_train = z_train['oh_label']
y_test = z_test['oh_label']



In [ ]:
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

In [ ]:
X_train.sample(1)

In [ ]:
X_test.sample(1)

# Model

In [ ]:
Loss_call_back = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='min')
_epoch = 50
_inputs =  len(X_train.columns)
_rows = len(X_train. index)
_batch = round(_rows/_epoch)

print(_epoch) 
print(_inputs)
print(_rows) 
print(_batch)

In [ ]:
obj = kt.Objective('val_loss', 'min')

tunner = kt.RandomSearch(MyHyperModel(), 
                         objective = obj, 
                         seed=42,
                         max_trials = 5, 
                         overwrite=True)

In [ ]:
tunner.search(np.asarray(X_train), 
              np.asarray(y_train),
              epochs=_epoch,
              validation_data=(np.asarray(X_test),np.asarray(y_test)), 
              batch_size= _batch,
              callbacks=[Loss_call_back])

In [ ]:
best_hps = tunner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

# Final model

In [ ]:
final_model = create_final_model(best_hps.get('units'), best_hps.get('learning_rate'))

In [ ]:
# history = final_model.fit(np.asarray(X_train), 
#                           np.asarray(y_train),
#                           epochs=_epoch,
#                           batch_size= _batch,
#                           validation_data = ( np.asarray(X_test), np.asarray(y_test) ),
#                           callbacks=[Loss_call_back])

history = final_model.fit(X_train, 
                          y_train,
                          epochs=_epoch,
                          batch_size= _batch,
                          validation_data = ( X_test, y_test ),
                          callbacks=[Loss_call_back])




In [ ]:
plot_loss(history)

In [ ]:
plot_precision(history, list(history.history)[1], list(history.history)[3])

# Save model

In [ ]:
# final_model.save('../models')